In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import joblib
from functools import reduce
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
# from skmultilearn.model_selection import MultiLabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
feat_breed = pd.read_pickle('../features/breed.pkl')
feat_breed_v2 = pd.read_pickle('../features/breed_v2.pkl')

feat_breed_top50 = pd.read_pickle('../features/breed_top50.pkl')
feat_breed_top50_v2 = pd.read_pickle('../features/breed_top50_v2.pkl')

feat_breed_group = pd.read_pickle('../features/breed_group.pkl')
feat_sub_breed = pd.read_pickle('../features/sub_breed.pkl')
feat_breed_type = pd.read_pickle('../features/breed_type.pkl')

feat_breed_pure_or_mix = pd.read_pickle('../features/breed_pure_or_mix.pkl')

feat_age = pd.read_csv('../features/age_with_id.csv')
feat_sex = pd.read_csv('../features/one_hot_encoded_sex_with_id.csv')

# Climate
feat_HotWheater = pd.read_csv('../features/HotMonths_with_dog_id.csv')
feat_residence = pd.read_csv('../features/Residence_with_dog_id.csv')

# Physical Activity
feat_pa_total_hours = pd.read_csv('../features/PhysicalActivity_total_hours.csv')
feat_pa_surface = pd.read_csv('../features/PhysicalActivity_surface.csv')

# Owner Demographics
feat_od_income = pd.read_csv('../features/od_income.csv')

feat_disease_input = pd.read_csv('../features/one_hot_encoded_disease_input.csv')
feat_disease_output_binary = pd.read_csv('../features/disease_output_binary.csv')
feat_disease_output = pd.read_csv('../features/disease_output.csv')

#Diet
feat_diet = pd.read_pickle('../features/diet_merged.pkl')

features_list = [
    #feat_breed,
    #feat_breed_v2,
    #feat_breed_top50,
    feat_breed_top50_v2,
    feat_breed_group,
    feat_sub_breed,
    feat_breed_type,
    feat_breed_pure_or_mix,
    feat_age,
    feat_sex,
    feat_HotWheater,
    feat_residence,
    feat_pa_total_hours,
    feat_pa_surface,
    feat_od_income,
    feat_diet
    #feat_disease_input,
    #feat_disease_output_binary,
    #feat_disease_output
]

In [3]:
exp1 = [feat_breed_group]
exp2 = [feat_breed_top50_v2]
exp3 = [feat_age]
exp4 = [feat_sex,feat_breed_type]
exp5 = [feat_sub_breed]
exp6 = [feat_breed_pure_or_mix]
exp7 = [feat_breed_top50_v2,feat_breed_group]
exp8 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex]
exp9 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_pa_surface,feat_pa_total_hours]
exp10 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_HotWheater,feat_residence]
exp11 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income]
exp12 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours]
exp13 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours]
exp14 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_diet]
exp15 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_diet]
exp16 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet]

In [4]:
experiments = {
    'exp1': exp1,
    'exp2': exp2,
    'exp3': exp3,
    'exp4': exp4,
    'exp5': exp5,
    'exp6': exp6,
    'exp7': exp7,
    'exp8': exp8,
    'exp9': exp9,
    'exp10': exp10,
    'exp11': exp11,
    'exp12': exp12,
    'exp13': exp13,
    'exp15': exp15,
    'exp16': exp16
}
results = {}

In [5]:
for exp_name, exp_features in experiments.items():
    # List of DataFrames to be merged
    list_input_features = features_list  # Add more DataFrames as needed

    # Merge DataFrames iteratively using reduce
    input_features = reduce(lambda left, right: pd.merge(left, right, on='dog_id'), exp_features)

    # Merge with disease output feature
    data = pd.merge(input_features, feat_disease_output_binary, on='dog_id')

    # Assuming 'data' is your DataFrame
    rows_with_nan = data[data.isna().any(axis=1)]
    columns_with_nan = data.columns[data.isna().any()].tolist()
    # Display the rows with NaN values
    rows_with_nan[columns_with_nan]
    # Separate features and labels
    X = data.drop(['dog_id'] + ['hs_health_conditions_' + condition for condition in [
        'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
        'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
        'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    ]], axis=1)

    y_columns = ['hs_health_conditions_' + condition for condition in [
        'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
        'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
        'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    ]]
    y = data[y_columns]

    # Convert y to a binary format
    y_binary = (y == 1)

    # Initialize the Naive Bayes model
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

    # Wrap the model with OneVsRestClassifier
    ovr_classifier = OneVsRestClassifier(model)

    # Initialize MultilabelStratifiedKFold
    n_splits = 5  # You can adjust the number of splits as needed
    ml_stratified_kfold = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    auc_scores_per_fold = []
    # Lists to store AUC scores and sample counts
    auc_scores_per_condition = {condition: [] for condition in y_columns}
    sample_counts_per_condition = {condition: [] for condition in y_columns}

    # Iterate through the splits
    for fold, (train_index, val_index) in enumerate(ml_stratified_kfold.split(X, y_binary)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[val_index]

        # Train the model
        ovr_classifier.fit(X_train, y_train)

        # Make predictions on the validation set
        y_pred_proba = ovr_classifier.predict_proba(X_val)

        # Calculate the AUC score for each disease
        auc_scores = [roc_auc_score(y_val[column], y_pred_proba[:, i]) for i, column in enumerate(y_val.columns)]

        auc_scores_per_fold.append(np.mean(auc_scores))
        # for i, condition in enumerate(y_columns):
        #     sample_count = y_val[condition].sum()  # Count of positive samples for the condition
        #     sample_counts_per_condition[condition].append(sample_count)

        # # print(f"\nFold {fold+1} AUC Scores for Diseases:")
        # for i, auc_score in enumerate(auc_scores, start=1):
        #     print(f"{y_val.columns[i-1]}: {auc_score}")
        #     auc_scores_per_condition[y_val.columns[i-1]].append(auc_score)
    # average auc score
    average_auc_score = np.mean(auc_scores_per_fold)
    results[exp_name] = average_auc_score

# print for each exp
for exp_name, auc_score in results.items():
    print(f"{exp_name}: Average AUC = {auc_score:.4f}")


exp1: Average AUC = 0.5626
exp2: Average AUC = 0.5776
exp3: Average AUC = 0.6665
exp4: Average AUC = 0.5591
exp5: Average AUC = 0.5668
exp6: Average AUC = 0.5227
exp7: Average AUC = 0.5823
exp8: Average AUC = 0.6968
exp9: Average AUC = 0.6960
exp10: Average AUC = 0.6932
exp11: Average AUC = 0.6927
exp12: Average AUC = 0.6944
exp13: Average AUC = 0.6009
exp15: Average AUC = 0.7012


In [30]:
# Calculate average AUC per condition
average_auc_per_condition = {
    condition: sum(auc_scores) / len(auc_scores) for condition, auc_scores in auc_scores_per_condition.items()
}
# Calculate overall average AUC weighted by the number of samples
overall_average_auc_sum = 0
for auc_scores, sample_counts in zip(auc_scores_per_condition.values(), sample_counts_per_condition.values()):
    weighted_sum = sum(auc * sample_count for auc, sample_count in zip(auc_scores, sample_counts))
    overall_average_auc_sum += weighted_sum / sum(sample_counts)

overall_average_auc = overall_average_auc_sum / len(auc_scores_per_condition)  # divide by the number of conditions
print('overall_average_auc',overall_average_auc)

In [ ]:
# Average AUC-score per condition
print("\n Average AUC Score per condition:")
for condition, avg_auc in average_auc_per_condition.items():
    print(f"{condition}: {avg_auc}")

# Get average AUC values as a list
average_auc_list = list(average_auc_per_condition.values())

# Calculate standard deviation of average AUC values
overall_auc_std = np.std(average_auc_list)

# Print average AUC scores
print("\nWeighted Average AUC Score:")
#print(f"Overall: {overall_average_auc}")
print(f"{overall_average_auc * 100:.2f}% ± {overall_auc_std * 100:.2f}%")

Save Model as .joblib

In [32]:
model_filepath = '../models/gradient_tree_boosting_with_breedtop50.joblib'
joblib.dump(ovr_classifier, model_filepath)

['../models/gradient_tree_boosting_with_breedtop50.joblib']

Test Model

In [33]:
loaded_model_filepath = '../models/gradient_tree_boosting_with_breedtop50.joblib'
loaded_ovr_classifier = joblib.load(loaded_model_filepath)

In [34]:
# Assuming X_val is your validation set DataFrame
# Select a random row (Datapoint) from X_val
random_index = np.random.choice(X_val.index)

Show input

In [35]:
random_data_point = X_val.loc[random_index]
random_data_point

breeds_American Pitbull Terrier              0
breeds_American Staffordshire Terrier        0
breeds_Australian Cattle Dog                 0
breeds_Australian Shepherd                   0
breeds_Basset Hound                          0
                                         ...  
grass_dirt                                True
gravel                                   False
sand                                     False
astroturf                                False
od_annual_income_range_usd                 5.0
Name: 8858, Length: 125, dtype: object

In [39]:
# Reshape the data point to a 2D array
random_data_point_reshaped = random_data_point.values.reshape(1, -1)

# Make predictions on the reshaped data point using the loaded model
new_data_predictions_proba = loaded_ovr_classifier.predict_proba(random_data_point_reshaped)
new_data_predictions_proba

array([[1.7784629e-02, 2.9939476e-02, 4.2124249e-02, 1.3108933e-01,
        1.1365089e-03, 2.4033024e-03, 7.0786998e-02, 1.1281752e-03,
        1.6732413e-02, 4.4480357e-03, 2.5812009e-02, 1.4144853e-03,
        6.3175685e-04, 8.8583899e-04, 2.8188192e-04, 3.5820755e-01,
        5.5941883e-02, 1.7371356e-01, 6.2181922e-03]], dtype=float32)